# Overview
This notebook demonstrates the proof of concept for the proposed daily notification scheme.

Assumptions:
- The strategy has been thoroughly backtested and ready for actual trading.
- Today is Nov 5, 2018, which is trading Day 0.

## Step 1: Initialize data (Day 0)
- Save the data to disk. This is so the script will only fetch the latest stock data.

In [1]:
import pandas as pd

from fastquant import backtest, get_stock_data

In [2]:
# Setup our variables
FILE_DIR = "jfc.csv"
SYMBOL = "JFC"
today = "2018-11-06"  # Fake today date for demo purposes

df = get_stock_data(SYMBOL, "2018-01-01", "2018-11-05")
df.to_csv(FILE_DIR)

## Step 2: Daily script calls (Day 1-2)
1. Fetch today's data
2. Load data from disk and append today's data
3. Run backtest with args `live=True` and `today=today`

### Demo Day 1: Nov 6, 2018 (buy)

In [3]:
def daily_fetch(file_dir, symbol, today):
    today_df = get_stock_data(symbol, today, today)

    # Retrieve saved historical data on disk and append new data
    # TODO: add checks if daily updates were broken
    df = pd.read_csv(file_dir, parse_dates=["dt"]).set_index("dt")
    # df = df.append(today_df)
    df = pd.concat([df, today_df], ignore_index=True)
    df.to_csv(file_dir)

    return df

In [4]:
today = "2018-11-06"  # Fake today date for demo purposes
df = daily_fetch(FILE_DIR, SYMBOL, today)

backtest(
    "smac", df, fast_period=15, slow_period=40, verbose=False, live=True, today=today
)

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 15
slow_period : 40
>>> Notif bot: BUY! <<<
Final Portfolio Value: 97652.8975
Final PnL: -2347.1
Time used (seconds): 0.05349397659301758
Optimal parameters: {'init_cash': 100000, 'buy_prop': 1, 'sell_prop': 1, 'commission': 0.0075, 'execution_type': 'close', 'live': True, 'today': '2018-11-06', 'notif_script_dir': False, 'symbol': '', 'fast_period': 15, 'slow_period': 40}
Optimal metrics: {'rtot': -0.023750856806590226, 'ravg': -0.00011418681157014532, 'rnorm': -0.028365016583377686, 'rnorm100': -2.8365016583377685, 'sharperatio': None, 'pnl': -2347.1, 'final_value': 97652.8975}


<IPython.core.display.Javascript object>

,init_cash,buy_prop,sell_prop,commission,execution_type,live,today,notif_script_dir,symbol,fast_period,slow_period,rtot,ravg,rnorm,rnorm100,sharperatio,pnl,final_value
0,100000,1,1,0.0075,close,True,2018-11-06,False,,15,40,-0.023751,-0.000114,-0.028365,-2.836502,None,-2347.1,97652.8975


- Notice the line that prints `>>> Notif bot: BUY! <<<`
- For this POC, it's just a print function, but it could be any script call.

### Demo Day 2: Nov 7, 2018 (hold)

In [5]:
today = "2018-11-07"  # Fake today date for demo purposes
df = daily_fetch(FILE_DIR, SYMBOL, today)

backtest(
    "smac", df, fast_period=15, slow_period=40, verbose=False, live=True, today=today
)

===Global level arguments===
init_cash : 100000
buy_prop : 1
sell_prop : 1
commission : 0.0075
===Strategy level arguments===
fast_period : 15
slow_period : 40
Final Portfolio Value: 98142.89749999999
Final PnL: -1857.1
Time used (seconds): 0.055661678314208984
Optimal parameters: {'init_cash': 100000, 'buy_prop': 1, 'sell_prop': 1, 'commission': 0.0075, 'execution_type': 'close', 'live': True, 'today': '2018-11-07', 'notif_script_dir': False, 'symbol': '', 'fast_period': 15, 'slow_period': 40}
Optimal metrics: {'rtot': -0.018745631612988572, 'ravg': -8.969201728702666e-05, 'rnorm': -0.02234886802384841, 'rnorm100': -2.234886802384841, 'sharperatio': None, 'pnl': -1857.1, 'final_value': 98142.89749999999}


,init_cash,buy_prop,sell_prop,commission,execution_type,live,today,notif_script_dir,symbol,fast_period,slow_period,rtot,ravg,rnorm,rnorm100,sharperatio,pnl,final_value
0,100000,1,1,0.0075,close,True,2018-11-07,False,,15,40,-0.018746,-0.00009,-0.022349,-2.234887,None,-1857.1,98142.8975


- Notice that there is no BUY or SELL notification
- TODO: Find a way to insert a HOLD notification. It's a good indicator that the cronjob is working as expected.

# Notes

When using in live trading, use:
```
from datetime import date

today = date.today().strftime("%Y-%m-%d")
```